The python environment I used for this was called My_New_Pymatgen in miniconda. The only package required was `pip install mp-api`

Note: The old vs new Materials Project API's have two different API Keys. Use the correct one. https://next-gen.materialsproject.org/api

In [ ]:
import pandas as pd
import os

filename = r'G:\My Drive\teaching\5540-6640 Materials Informatics\apikey.txt'

def get_file_contents(filename):
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)


Sparks_API = get_file_contents(filename)

In [ ]:
import pymatgen.core as mg
si = mg.Element("Si")
print('Silicon has atomic mass of:', si.atomic_mass)

The API for Materials Project recently (2022) was updated. You can read about the differences between new and old with API Key and install instructions for each.
https://docs.materialsproject.org/downloading-data/differences-between-new-and-legacy-api

For this class, let's use the new API which you can read about here https://api.materialsproject.org/docs

To install, in miniconda in your My_Pymatgen environment run the command 'pip install mp-api'

First, I had to update pydantic with the following code 'pip install pydantic==2.0'



In [ ]:
#this works with the old legacy MPRester
from mp_api.client import MPRester

with MPRester(Sparks_API) as mpr:
    list_ids = mpr.get_materials_ids("TaC")
    print('All compounds in the Ta-C system are: ', mpr.get_materials_ids('Ta-C'))

Or we can pull data from a specific materials project id

In [ ]:
for i_d in list_ids:
    with MPRester(Sparks_API) as mpr:
        structure = mpr.get_structure_by_material_id(i_d)
        print(structure)


with MPRester(Sparks_API) as mpr:
    structure = mpr.get_structure_by_material_id('mp-1086')
    print(structure)
    


How do we do queries though? What if we want to find all carbides having either Ta, Nb, or W?
We need to use the MPRester.summary.search method!
https://docs.materialsproject.org/downloading-data/using-the-api/querying-data

By default it grabs ALL the property data available, but you can also tell it to only grab a few specific fields. Some students report errors if you leave the fields blank and found that it worked if you provided fields. 


In [ ]:
mpr = MPRester(Sparks_API)
#grab all the data
docs = mpr.summary.search(elements=['Si','O'],band_gap=(0.85,1))
print(docs[0])
#just grab a few specific fields
docs = mpr.summary.search(elements=['Si','O'],band_gap=(0.85,1),fields=["material_id","density","symmetry"])
print(docs[0])
#call up a specific field for a entry as follows
print('The chemical system is',docs[0].density)


In [ ]:
with MPRester(Sparks_API) as mpr:
    docs = mpr.summary.search(material_ids=["mp-149"], fields=["symmetry"])
    structure = docs[0].symmetry
    # -- Shortcut for a single Materials Project ID:
    structure = mpr.get_structure_by_material_id("mp-149")
    print(structure)

There are lots of examples that you can peruse here
https://docs.materialsproject.org/downloading-data/using-the-api/examples

In [ ]:
#Find all Materials Project IDs for entries with dielectric data
from mp_api.client import MPRester
from emmet.core.summary import HasProps

with MPRester(Sparks_API) as mpr:
    docs = mpr.summary.search(
        has_props = [HasProps.dielectric], fields=["material_id"]
    )
    mpids = [doc.material_id for doc in docs]

In [ ]:
#Calculation (task) IDs and types for silicon (mp-149)
from mp_api.client import MPRester

with MPRester(Sparks_API) as mpr: 
    docs = mpr.materials.search(material_ids=["mp-149"], fields=["calc_types"])
    task_ids = docs[0].calc_types.keys()
    task_types = docs[0].calc_types.values() 
    # -- Shortcut for a single Materials Project ID:
    task_ids = mpr.get_task_ids_associated_with_material_id("mp-149")
    print(task_ids)

In [ ]:
#find Band gaps for all materials containing only Si and O
from mp_api.client import MPRester

with MPRester(Sparks_API) as mpr:
    docs = mpr.summary.search(
        chemsys="Si-O", fields=["material_id", "band_gap"]
    )
    mpid_bgap_dict = {doc.material_id: doc.band_gap for doc in docs}
    print(mpid_bgap_dict)

In [ ]:
#Chemical formulas for all materials containing at least Si and O
from mp_api.client import MPRester

with MPRester(Sparks_API) as mpr:
    docs = mpr.summary.search(
        elements=["Si", "O"], fields=["material_id", "band_gap", "formula_pretty"]
    )
    mpid_formula_dict = {
        doc.material_id: doc.formula_pretty for doc in docs
    }

In [ ]:
#Stable materials (on the GGA/GGA+U hull) with large band gaps (>3eV)
from mp_api.client import MPRester

with MPRester(Sparks_API) as mpr:
    docs = mpr.summary.search(
        band_gap=(3, None), is_stable=True, fields=["material_id"]
    )
    stable_mpids = [doc.material_id for doc in docs]
    
    ## -- Alternative directly using energy above hull:
    docs = mpr.summary.search(
        band_gap=(3, None), energy_above_hull=(0, 0), fields=["material_id"]
    )
    stable_mpids = [doc.material_id for doc in docs]

In [ ]:
#Band structures for silicon (mp-149)
from mp_api.client import MPRester
from emmet.core.electronic_structure import BSPathType

with MPRester(Sparks_API) as mpr:
    # -- line-mode, Setyawan-Curtarolo (default):
    bs_sc = mpr.get_bandstructure_by_material_id("mp-149")
    
    # -- line-mode, Hinuma et al.:
    bs_hin = mpr.get_bandstructure_by_material_id("mp-149", path_type=BSPathType.hinuma)

    # -- line-mode, Latimer-Munro:
    bs_hin = mpr.get_bandstructure_by_material_id("mp-149", path_type=BSPathType.latimer_munro)
    
    # -- uniform:
    bs_uniform = mpr.get_bandstructure_by_material_id("mp-149", line_mode=False)                            

In [ ]:
#Density of states for silicon (mp-149)
from mp_api.client import MPRester

with MPRester(Sparks_API) as mpr:
    dos = mpr.get_dos_by_material_id("mp-149")

In [ ]:
from mp_api.client import MPRester
from emmet.core.thermo import ThermoType

with MPRester(Sparks_API) as mpr:
    
    # -- GGA/GGA+U/R2SCAN mixed phase diagram
    pd = mpr.thermo.get_phase_diagram_from_chemsys(chemsys="Li-Fe-O", 
                                                   thermo_type=ThermoType.GGA_GGA_U_R2SCAN)
    
    # -- GGA/GGA+U mixed phase diagram
    pd = mpr.thermo.get_phase_diagram_from_chemsys(chemsys="Li-Fe-O", 
                                                   thermo_type=ThermoType.GGA_GGA_U)
                                                   
    # -- R2SCAN only phase diagram
    pd = mpr.thermo.get_phase_diagram_from_chemsys(chemsys="Li-Fe-O", 
                                                   thermo_type=ThermoType.R2SCAN)


In [ ]:
from mp_api.client import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter

with MPRester(Sparks_API) as mpr:

    # Obtain only corrected GGA and GGA+U ComputedStructureEntry objects
    entries = mpr.get_entries_in_chemsys(elements=["Li", "Fe", "O"], 
                                         additional_criteria={"thermo_types": ["GGA_GGA+U"]}) 
    # Construct phase diagram
    pd = PhaseDiagram(entries)
    
    # Plot phase diagram
    PDPlotter(pd).get_plot()
    

In [ ]:
#let's show the phase diagram. I had to first install nbformat 'pip install --upgrade nbformat'
PDPlotter(pd).get_plot()

# Now you try it!